In [3]:
# Install torch with cuda ver11.8
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 991.0 kB/s eta 0:45:18
     ---------------------------------------- 0.0/2.7 GB 1.1 MB/s eta 0:42:44
     ---------------------------------------- 0.0/2.7 GB 1.3 MB/s eta 0:34:08
     ---------------------------------------- 0.0/2.7 GB 1.6 MB/s eta 0:28:06
     ---------------------------------------- 0.0/2.7 GB 1.6 MB/s eta 0:27:28
     ---------------------------------------- 0.0/2.7 GB 1.7 MB/s eta 0:25:57
     ---------------------------------------- 0.0/2.7 GB 2.3 MB/s eta 0:19:52
     ---------------------------------------- 0.0/2.7 GB 2.6 MB/s eta 0:17:35
     ---------------------------------------- 0.0/2.7 GB 3.3 MB/s eta 0:13:41
     ---------------------------------------- 0.0/2.7 GB 4.5 MB/s eta 0:09:58
     ---------------------------------------- 0.0/2.7 GB 6.0 MB/s eta 0:07:26
     -----


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import torch
from lips import get_root_path
from lips.dataset.airfransDataSet import download_data
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark

In [2]:
print(torch.cuda.is_available())

True


In [3]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = 'dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

In [4]:
BENCH_CONFIG_PATH = "C:/Users/hoang/OneDrive/Desktop/NeuIPS_competition/LIPS/lips/benchmark/confAirfoil.ini"
SIM_CONFIG_PATH = "C:/Users/hoang/OneDrive/Desktop/NeuIPS_competition/LIPS/configurations/airfoil/simulators/torch_fc.ini"

In [5]:
SIM_CONFIG_PATH

'C:/Users/hoang/OneDrive/Desktop/NeuIPS_competition/LIPS/configurations/airfoil/simulators/torch_fc.ini'

In [6]:
if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

In [7]:
benchmark=AirfRANSBenchmark(benchmark_path = DIRECTORY_NAME,
                            config_path = BENCH_CONFIG_PATH,
                            benchmark_name = BENCHMARK_NAME,
                            log_path = LOG_PATH)
benchmark.load(path=DIRECTORY_NAME)

Loading dataset (task: reynolds, split: test): 100%|██████████| 496/496 [03:51<00:00,  2.14it/s]


In [8]:
for datasetName,dataset in zip(["Train","Test","OOD"],[benchmark.train_dataset,benchmark._test_dataset,benchmark._test_ood_dataset]):
    print("%s dataset: "%datasetName)
    print("\t Number of simulations:", len(dataset.get_simulations_sizes()))

Train dataset: 
	 Number of simulations: 103
Test dataset: 
	 Number of simulations: 200
OOD dataset: 
	 Number of simulations: 496


In [9]:
print("Benchmark name: ", benchmark.config.section_name)

Benchmark name:  Case1


In [10]:
print("Input attributes (features): ", benchmark.config.get_option("attr_x"))

Input attributes (features):  ('x-position', 'y-position', 'x-inlet_velocity', 'y-inlet_velocity', 'distance_function', 'x-normals', 'y-normals')


In [11]:
print("Output attributes (targets): ", benchmark.config.get_option("attr_y"))

Output attributes (targets):  ('x-velocity', 'y-velocity', 'pressure', 'turbulent_viscosity')


In [12]:
print("Evaluation criteria: ")
criteria = benchmark.config.get_option("eval_dict")
for criteriaCategory,criteriaNames in criteria.items():
    print("\t Category %s: " %criteriaCategory," ",criteriaNames)

Evaluation criteria: 
	 Category ML:    ['MSE', 'MAPE']
	 Category Physics:    ['spearman_correlation_drag', 'spearman_correlation_lift', 'mean_relative_drag', 'std_relative_drag', 'mean_relative_lift', 'std_relative_lift']


In [13]:
print("train dataset: ", benchmark.train_dataset)
print("test dataset: ", benchmark._test_dataset )
print("test dataset: ", benchmark._test_ood_dataset )

train dataset:  Instance of AirfRANSDataSet
Dataset name: train
	x-position
		 Data size: (18515415,)
	y-position
		 Data size: (18515415,)
	x-inlet_velocity
		 Data size: (18515415,)
	y-inlet_velocity
		 Data size: (18515415,)
	distance_function
		 Data size: (18515415,)
	x-normals
		 Data size: (18515415,)
	y-normals
		 Data size: (18515415,)
	x-velocity
		 Data size: (18515415,)
	y-velocity
		 Data size: (18515415,)
	pressure
		 Data size: (18515415,)
	turbulent_viscosity
		 Data size: (18515415,)

test dataset:  Instance of AirfRANSDataSet
Dataset name: test
	x-position
		 Data size: (35849332,)
	y-position
		 Data size: (35849332,)
	x-inlet_velocity
		 Data size: (35849332,)
	y-inlet_velocity
		 Data size: (35849332,)
	distance_function
		 Data size: (35849332,)
	x-normals
		 Data size: (35849332,)
	y-normals
		 Data size: (35849332,)
	x-velocity
		 Data size: (35849332,)
	y-velocity
		 Data size: (35849332,)
	pressure
		 Data size: (35849332,)
	turbulent_viscosity
		 Data size: (

In [14]:
from lips.augmented_simulators.torch_models.fully_connected import TorchFullyConnected
from lips.augmented_simulators.torch_simulator import TorchSimulator
from lips.dataset.scaler.standard_scaler_iterative import StandardScalerIterative


chunk_sizes=benchmark.train_dataset.get_simulations_sizes()
no_norm_x=benchmark.train_dataset.get_no_normalization_axis_indices()
scalerParams={"chunk_sizes":chunk_sizes,"no_norm_x":no_norm_x}

augmented_simulator = TorchSimulator(name="torch_fc",
                                     model=TorchFullyConnected,
                                     scaler=StandardScalerIterative,
                                     log_path="log_benchmark",
                                     device="cuda:0",
                                     bench_config_path=BENCH_CONFIG_PATH,
                                     bench_config_name=BENCHMARK_NAME,
                                     sim_config_path=SIM_CONFIG_PATH,
                                     sim_config_name="DEFAULT",
                                     scalerParams=scalerParams
                                    )

In [17]:
augmented_simulator.train(train_dataset=benchmark.train_dataset, 
                          epochs=30, 
                          train_batch_size=1024,
                          pin_memory=True, 
                          non_blocking=True, 
                          num_workers=4, 
                          )

In [ ]:
import time

start_test = time.time()
fc_metrics_test = benchmark.evaluate_simulator(dataset="test",augmented_simulator=augmented_simulator,eval_batch_size=256000,num_workers=16 )
test_evaluation_time = time.time() - start_test
print(test_evaluation_time)
test_mean_simulation_time = test_evaluation_time/len(benchmark._test_dataset.get_simulations_sizes())

In [ ]:
start_test_ood = time.time()
fc_metrics_test_ood = benchmark.evaluate_simulator(dataset="test_ood",augmented_simulator=augmented_simulator,eval_batch_size=256000,num_workers=16 )
test_ood_evaluation_time = time.time() - start_test_ood
test_ood_mean_simulation_time = test_ood_evaluation_time/len(benchmark._test_ood_dataset.get_simulations_sizes())
print(test_ood_evaluation_time)

In [ ]:
ML_metrics = "ML"
dataset_name = "test"
print("Fully Connected Augmented Simulator")
print(f"Dataset : {dataset_name}")
print("{:<10} : {}".format("ML metrics", fc_metrics_test[dataset_name][ML_metrics]))

In [ ]:
physic_compliances = "Physics"
dataset_name = "test"
physical_metrics = fc_metrics_test[dataset_name][physic_compliances]
print(physical_metrics)

In [ ]:
ood_ml_metrics = fc_metrics_test_ood["test_ood"][ML_metrics]
ood_physical_metrics = fc_metrics_test_ood["test_ood"][physic_compliances]
print(ood_ml_metrics)
print(ood_physical_metrics)

In [ ]:
thresholds={"x-velocity":(0.1,0.2,"min"),
            "y-velocity":(0.1,0.2,"min"),
            "pressure":(0.02,0.1,"min"),
            "pressure_surfacic":(0.08,0.2,"min"),
            "turbulent_viscosity":(0.5,1.0,"min"),
            "mean_relative_drag":(1.0,10.0,"min"),
            "mean_relative_lift":(0.2,0.5,"min"),
            "spearman_correlation_drag":(0.5,0.8,"max"),
            "spearman_correlation_lift":(0.94,0.98,"max")           
    }

In [ ]:
configuration={
        "coefficients":{"ML":0.4,"OOD":0.3,"Physics":0.3},
        "ratioRelevance":{"Speed-up":0.25,"Accuracy":0.75},
        "valueByColor":{"g":2,"o":1,"r":0},
        "maxSpeedRatioAllowed":10000,
        "reference_mean_simulation_time":1500
    }

In [ ]:
ml_metrics = fc_metrics_test["test"][ML_metrics]["MSE_normalized"]
ml_metrics["pressure_surfacic"] = fc_metrics_test["test"][ML_metrics]["MSE_normalized_surfacic"]["pressure"]

phy_variables_to_keep = ["mean_relative_drag","mean_relative_lift","spearman_correlation_drag","spearman_correlation_lift"]
phy_metrics = {phy_variable:fc_metrics_test["test"][physic_compliances][phy_variable] for phy_variable in phy_variables_to_keep}

ml_ood_metrics = fc_metrics_test_ood["test_ood"][ML_metrics]["MSE_normalized"]
ml_ood_metrics["pressure_surfacic"] = fc_metrics_test_ood["test_ood"][ML_metrics]["MSE_normalized_surfacic"]["pressure"]
phy_ood_metrics = {phy_variable:fc_metrics_test_ood["test_ood"][physic_compliances][phy_variable] for phy_variable in phy_variables_to_keep}
ood_metrics = {**ml_ood_metrics,**phy_ood_metrics}

all_metrics={
    ML_metrics:ml_metrics,
    physic_compliances:phy_metrics,
    "OOD":ood_metrics
}

print(all_metrics)

In [ ]:
reference_mean_simulation_time=configuration["reference_mean_simulation_time"]
speedUp={
        ML_metrics:reference_mean_simulation_time/test_mean_simulation_time,
         "OOD":reference_mean_simulation_time/test_ood_mean_simulation_time
        }

In [ ]:
import math
def compute_global_score(metrics,speedUp,configuration,thresholds):
    accuracyResults=dict()
    for subcategoryName, subcategoryVal in metrics.items():
        accuracyResults[subcategoryName]=[]
        for variableName, variableError in subcategoryVal.items():
            thresholdMin,thresholdMax,evalType=thresholds[variableName]
            if evalType=="min":
                if variableError<thresholdMin:
                    accuracyEval="g"
                elif thresholdMin<variableError<thresholdMax:
                    accuracyEval="o"
                else:
                    accuracyEval="r"
            elif evalType=="max":
                if variableError<thresholdMin:
                    accuracyEval="r"
                elif thresholdMin<variableError<thresholdMax:
                    accuracyEval="o"
                else:
                    accuracyEval="g"
    
            accuracyResults[subcategoryName].append(accuracyEval)

    print("accuracyResults: ",accuracyResults)
    
    coefficients=configuration["coefficients"]
    ratioRelevance=configuration["ratioRelevance"]
    valueByColor=configuration["valueByColor"]
    maxSpeedRatioAllowed=configuration["maxSpeedRatioAllowed"]
    mlSubscore=0
    accuracyMaxPoints=ratioRelevance["Accuracy"]
    accuracyResult=sum([valueByColor[color] for color in accuracyResults["ML"]])
    accuracyResult=accuracyResult*accuracyMaxPoints/(len(accuracyResults["ML"])*max(valueByColor.values()))
    mlSubscore+=accuracyResult
    speedUpMaxPoints=ratioRelevance["Speed-up"]
    speedUpResult=max(0,min(math.log10(speedUp["ML"])/math.log10(maxSpeedRatioAllowed),1))
    speedUpResult=speedUpResult*speedUpMaxPoints
    mlSubscore+=speedUpResult

    accuracyResult=sum([valueByColor[color] for color in accuracyResults["Physics"]])
    accuracyResult=accuracyResult/(len(accuracyResults["Physics"])*max(valueByColor.values()))
    physicsSubscore=accuracyResult

    oodSubscore=0
    accuracyMaxPoints=ratioRelevance["Accuracy"]
    accuracyResult=sum([valueByColor[color] for color in accuracyResults["OOD"]])
    accuracyResult=accuracyResult*accuracyMaxPoints/(len(accuracyResults["OOD"])*max(valueByColor.values()))
    oodSubscore+=accuracyResult
    
    speedUpMaxPoints=ratioRelevance["Speed-up"]
    speedUpResult=max(0,min(math.log10(speedUp["OOD"])/math.log10(maxSpeedRatioAllowed),1))
    speedUpResult=speedUpResult*speedUpMaxPoints
    oodSubscore+=speedUpResult

    globalScore=100*(coefficients["ML"]*mlSubscore+coefficients["Physics"]*physicsSubscore+coefficients["OOD"]*oodSubscore)
    return globalScore

In [ ]:
globalScore = compute_global_score(all_metrics,speedUp,configuration,thresholds)
print(globalScore)